In [94]:
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set, scale_tuple, scale_list, scale_changer
import matplotlib.pyplot as plt
from matplotlib import rc
from typing import Union, Tuple, List
from pyomo.environ import ConcreteModel, Var
from pyomo.contrib.iis import write_iis
from pyomo.environ import value as pyo_val

In [95]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 4  # Number of execution scenarios                     (chi)
M = 1e4  # Big M

# Define temporal scales
time_scales = TemporalScale(discretization_list=[_exec_scenarios, _time_intervals])

network_scale_level = 0
scheduling_scale_level = 1
annualization_factor = 1/len(time_scales.scale[network_scale_level])
scale_iter_list = time_scales.scale_iter(scale_level=len(time_scales.scale)-1)

In [96]:
scale_iter_list

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6)]

In [97]:
def formulate_model(start_time:tuple, end_time:tuple, info_dict:dict=None):

    scales = TemporalScale(discretization_list=[_exec_scenarios, _time_intervals], start_zero=start_time, end_zero=end_time)
    
    loc6_varying_factor = pandas.DataFrame(data={'com1_process': [1]*_exec_scenarios})
    # print(loc6_varying_factor)
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=75, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
    com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
    com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
    com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
    com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 125
    min_process_capacity = 0.01
    
    com1_procure = Process(name='procure com1', prod_max=125, conversion={com1_pur: -1, com1_in: 1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Procure com1')
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Sell com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 1')
    com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 2')
    com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=com1_process_capacity, conversion={com1_loc3_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 3')
    com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=com1_process_capacity, conversion={com1_loc4_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 4')
    com1_receive_loc5 = Process(name='com1_receive_loc5', prod_max=com1_process_capacity, conversion={com1_loc5_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 5')
    com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=com1_process_capacity, conversion={com1_loc6_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 6')
    com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=com1_process_capacity, conversion={com1_loc7_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Commodity 1 received from location 7')
    
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=min_process_capacity, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    # com1_process = Process(name='com1_process', prod_max=prod_max, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=prod_min, rate_max=rate_max, varying=[VaryingProcess.DETERMINISTIC_CAPACITY], label='Process the commodity through the location')
    
    com1_store = Process(name='com1_store', prod_max=com1_process_capacity, capex=500, vopex=5, store_min=0.01, store_max= 100, prod_min=min_process_capacity, label="Storage capacity of upto 100 units", storage=com1_in, storage_cost=0.02, storage_capex=50)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 1')
    com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 2')
    com1_loc3_send = Process(name='com1_loc3_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc3_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 3')
    com1_loc4_send = Process(name='com1_loc4_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc4_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 4')
    com1_loc5_send = Process(name='com1_loc5_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc5_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 5')
    com1_loc6_send = Process(name='com1_loc6_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc6_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 6')
    com1_loc7_send = Process(name='com1_loc7_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc7_out:1}, capex=0.01, vopex=0.01, prod_min=min_process_capacity, label='Send commodity one from location 7')
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_receive_loc2, com1_receive_loc3, com1_process, com1_store, com1_loc1_send}, label="Location 1", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc2 = Location(name='loc2', processes={com1_receive_loc1, com1_receive_loc4, com1_receive_loc5, com1_process, com1_store, com1_loc2_send}, label="Location 2", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc3 = Location(name='loc3', processes={com1_receive_loc1, com1_receive_loc4, com1_process, com1_store, com1_loc3_send}, label="Location 3", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc4 = Location(name='loc4', processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc6, com1_receive_loc5, com1_receive_loc7, com1_process, com1_store, com1_loc4_send}, label="Location 4", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc5 = Location(name='loc5', processes={com1_receive_loc2, com1_receive_loc4, com1_receive_loc7, com1_process, com1_store, com1_loc5_send, com1_sell}, label="Location 5", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    loc6 = Location(name='loc6', processes={com1_procure, com1_receive_loc4, com1_process, com1_store, com1_loc6_send}, label="Location 6", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0, capacity_factor={com1_process: loc6_varying_factor[['com1_process']]})
    
    loc7 = Location(name='loc7', processes={com1_receive_loc4, com1_receive_loc5, com1_process, com1_store, com1_loc7_send}, label="Location 7", scales=scales, demand_scale_level=1, capacity_scale_level=0, availability_scale_level=0)
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    truck_cap12 = 70
    truck_cap13 = 30
    truck_cap24 = 50
    truck_cap25 = 30
    truck_cap34 = 30
    truck_cap45 = 100
    truck_cap47 = 40
    truck_cap64 = 50
    truck_cap75 = 40
    
    truck_capmin = 0.01
    
    truck12 = Transport(name='truck12', resources={com1_loc1_out}, trans_max=truck_cap12, label='Truck from location 1 to 2', capex=0.5, vopex=0.05, trans_min=truck_capmin, speed=50)
    truck21 = Transport(name='truck21', resources={com1_loc2_out}, trans_max=truck_cap12, label='Truck from location 2 to 1', capex=0.0001, vopex=0.05, trans_min=truck_capmin, speed=50)
    
    truck13 = Transport(name='truck13', resources={com1_loc1_out}, trans_max=truck_cap13, label='Truck from location 1 to 3', capex=0.3, vopex=0.03, trans_min=truck_capmin, speed=50)
    truck31 = Transport(name='truck31', resources={com1_loc3_out}, trans_max=truck_cap13, label='Truck from location 3 to 1', capex=0.0001, vopex=0.03, trans_min=truck_capmin, speed=50)
    
    truck24 = Transport(name='truck24', resources={com1_loc2_out}, trans_max=truck_cap24, label='Truck from location 2 to 4', capex=0.5, vopex=0.05, trans_min=truck_capmin, speed=50)
    truck42 = Transport(name='truck42', resources={com1_loc4_out}, trans_max=truck_cap24, label='Truck from location 4 to 2', capex=0.0001, vopex=0.05, trans_min=truck_capmin, speed=50)
    
    truck25 = Transport(name='truck25', resources={com1_loc2_out}, trans_max=truck_cap25, label='Truck from location 2 to 5', capex=0.3, vopex=0.03, trans_min=truck_capmin, speed=50)
    truck52 = Transport(name='truck52', resources={com1_loc5_out}, trans_max=truck_cap25, label='Truck from location 5 to 2', capex=0.0001, vopex=0.03, trans_min=truck_capmin, speed=50)
    
    truck34 = Transport(name='truck34', resources={com1_loc3_out}, trans_max=truck_cap34, label='Truck from location 3 to 4', capex=0.2, vopex=0.02, trans_min=truck_capmin, speed=50)
    truck43 = Transport(name='truck43', resources={com1_loc4_out}, trans_max=truck_cap34, label='Truck from location 4 to 3', capex=0.0001, vopex=0.02, trans_min=truck_capmin, speed=50)
    
    truck45 = Transport(name='truck45', resources={com1_loc4_out}, trans_max=truck_cap45, label='Truck from location 4 to 5', capex=1, vopex=0.1, trans_min=truck_capmin, speed=50)
    truck54 = Transport(name='truck54', resources={com1_loc5_out}, trans_max=truck_cap45, label='Truck from location 5 to 4', capex=0.0001, vopex=0.1, trans_min=truck_capmin, speed=50)
    
    truck47 = Transport(name='truck47', resources={com1_loc4_out}, trans_max=truck_cap47, label='Truck from location 4 to 7', capex=0.4, vopex=0.04, trans_min=truck_capmin, speed=50)
    truck74 = Transport(name='truck74', resources={com1_loc7_out}, trans_max=truck_cap47, label='Truck from location 7 to 4', capex=0.0001, vopex=0.04, trans_min=truck_capmin, speed=50)
    
    truck64 = Transport(name='truck64', resources={com1_loc6_out}, trans_max=truck_cap64, label='Truck from location 6 to 4', capex=0.5, vopex=0.05, trans_min=truck_capmin, speed=50)
    truck46 = Transport(name='truck46', resources={com1_loc4_out}, trans_max=truck_cap64, label='Truck from location 4 to 6', capex=0.0001, vopex=0.05, trans_min=truck_capmin, speed=50)
    
    truck75 = Transport(name='truck75', resources={com1_loc7_out}, trans_max=truck_cap75, label='Truck from location 7 to 5', capex=0.4, vopex=0.04, trans_min=truck_capmin, speed=50)
    truck57 = Transport(name='truck57', resources={com1_loc5_out}, trans_max=truck_cap75, label='Truck from location 5 to 7', capex=0.0001, vopex=0.04, trans_min=truck_capmin, speed=50)
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================
    
    # transport_matrix = [
    #     [[], [truck50_12], [truck30_13], [], [], [], []],  # source: location 1
    #     [[truck50_21], [], [], [truck50_24], [truck30_25], [], []],  # source: location 2
    #     [[truck30_31], [], [], [truck20_34], [], [], []],  # source: location 3
    #     [[], [truck50_42], [truck20_43], [], [truck100_45], [truck50_46], [truck40_47]],  # source: location 4
    #     [[], [truck30_52], [], [truck100_54], [], [], [truck40_57]],  # source: location 5
    #     [[], [], [], [truck50_64], [], [], []],  # source: location 6
    #     [[], [], [], [truck40_74], [truck40_75], [], []]  # source: location 7
    # ]
    
    transport_matrix = [
        [[], [truck12], [truck13], [], [], [], []],  # source: location 1
        [[truck21], [], [], [truck24], [truck25], [], []],  # source: location 2
        [[truck31], [], [], [truck34], [], [], []],  # source: location 3
        [[], [truck42], [truck43], [], [truck45], [truck46], [truck47]],  # source: location 4
        [[], [truck52], [], [truck54], [], [], [truck57]],  # source: location 5
        [[], [], [], [truck64], [], [], []],  # source: location 6
        [[], [], [], [truck74], [truck75], [], []]  # source: location 7
    ]
    
    distance_matrix = [
        [0, 55, 196, M, M, M, M],
        [55, 0, M, 163, 112, M, 134],
        [196, M, 0, 63, M, M, M],
        [M, 163, 63, 0, 95, 117, 88],
        [M, 112, M, 95, 0, M, 134],
        [M, M, M, 117, M, 0, M],
        [M, 134, M, 88, 134, M, 0]
    ]
    
    locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7]
    
    sources = locset
    sinks = locset
    
    network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix)
    
    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 50
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc5 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc5 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name=f'scenario', scales=scales, scheduling_scale_level=1, network_scale_level=0, purchase_scale_level=1, availability_scale_level=0, demand_scale_level=1, capacity_scale_level=0, network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Baseline Scenario', annualization_factor=annualization_factor)
    
    # ======================================================================================================================
    # Declare model
    # ======================================================================================================================
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.TRANSPORT, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK, Constraints.MODE},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    return scenario, problem_mincost

In [98]:
def fix_variables(model2fix: ConcreteModel, current_time_idx:tuple, initial_dict: dict,):
    states = ['P', 'B', 'C', 'S', 'R', 'Inv', 'P_m', 'P_material_m', 'Cap_P', 'Cap_S', 'Capex_process', 'Capex_storage', 'Inv_cost', 'X_F', 'Cap_F', 'Exp', 'Capex_transport', 'X_P', 'X_S', 'X_P_m', 'Cap_P_m', 'X_P_mm', 'Demand_penalty', 'Demand_penalty_cost']
    
    model2fix_vars = {v.name: v for v in model2fix.component_objects(Var) if v.name in states}
    
    for name, var in model2fix_vars.items():
        var_key_list = [k for k in var.keys()]
        updated_key_list = list()
        
        if var_key_list and var_key_list!= [None]:
            if isinstance(var_key_list[0], int):
                # updated_key_list = [i for i in var_key_list if tuple((i,)) < current_time_idx[:network_scale_level+1]] 
                continue
            elif isinstance(var_key_list[0], tuple) and ('location' not in name and 'network' not in name):
                num_integers = sum(1 for elem in var_key_list[0] if isinstance(elem, int))
                scale_length = min(len(time_scales.scale), num_integers)                
                updated_key_list = [k for k in var.keys()
                                    if tuple(k[-scale_length:]) < current_time_idx[:scale_length] or (tuple(k[-scale_length:]) == current_time_idx[:scale_length] and current_time_idx[1:]!=(0,)*(len(current_time_idx)-1) and tuple(k[-scale_length:]) != current_time_idx)]
                
            for key in updated_key_list:
                if key in initial_dict[name] and initial_dict[name][key] is not None:
                    if not var[key].fixed:
                        var[key].fix(initial_dict[name][key])
                    # var[key].fixed = True
                    # var[key] = initial_dict[name][key]
    print(f"Variables fixed for time {current_time_idx}")

In [99]:
def print_fixed_variables(instance:ConcreteModel):    
    for v in instance.component_objects(Var):
        for index in v:
            if v[index].fixed:
                print(f"{v.name}[{index}]: {v[index].value}")

def get_fixed_variables(instance:ConcreteModel):
    return {v.name: {index: v[index].value for index in v if v[index].fixed} for v in instance.component_objects(Var)}


In [100]:
# def compile_results(close_loop_results:dict=None, time_dict:dict=None, start:int=None, end:int=None):
#     final_results = {key: {} for key in close_loop_results[time_dict[start]]}
#     for t in range(start,end+1):
#         for name in close_loop_results[time_dict[t]]:
#             for key, value in name.items():
#                 if key:
#                     if isinstance(key, int):
#                         
#                 num_integers = sum(1 for elem in key)
#     
#     return final_results    

In [101]:
model_record = dict()
scenario_record = dict()
results_record = dict()

In [102]:
def supply_chain_controller(time_ind:List, start_at:Union[int, Tuple]=0, end_at:Union[int, Tuple]=None, initialize_dict:dict=None, p_horizon:int=1, c_horizon:int=1, lookback:int=0):
    
    end_at = end_at or len(time_ind)-1
    
    close_loop_results = dict()
    if c_horizon > p_horizon:
        raise ValueError('c_horizon must be less than or equal to p_horizon')
    
    if isinstance(start_at, Tuple):
        if start_at not in time_ind:
            raise ValueError('Incorrect start time value')
        start_idx = time_ind.index(start_at)
    else:
        start_idx = start_at
        
    if isinstance(end_at, Tuple):
        if end_at not in time_ind:
            raise ValueError('Incorrect end time value')
        end_idx = time_ind.index(end_at)
    else:
        end_idx = end_at
    
    if end_idx < start_idx:
        raise ValueError('Incorrect start and end time values')

    # print(f"Starting Index: {start_idx}; Starting Time: {start_time}")
    # print(f"Ending Index: {end_idx}; Ending Time: {end_time}")
    
    
    for t in range(start_idx, end_idx+1, c_horizon):
        start_time = max(0, t-lookback)
        end_time = min(t+p_horizon, end_idx)
        scenario, model = formulate_model(start_time=time_ind[start_time], end_time=time_ind[end_time])
        scenario_record[time_ind[t]] = scenario
        model_record[time_ind[t]] = model
        
        if time_ind[t-1] in close_loop_results:
            fix_variables(model2fix=model, initial_dict=close_loop_results[time_ind[t-1]], current_time_idx=time_ind[t])
        
        result = solve(scenario=scenario, instance=model, solver='gurobi', name='MILP')
        model_vars = model.component_map(ctype=Var)
        close_loop_results[time_ind[t]] = {i: model_vars[i].extract_values() for i in model_vars.keys()}
        results_record[time_ind[t]] = result

        print(f'\n################################## Finished solving problem for time period {time_ind[t]} out of {time_ind[-1]} ##################################\n')
        # final_results = compile_results(close_loop_results=close_loop_results, time_dict = time_ind, start = start_idx, end = end_idx)
    
    return close_loop_results

In [103]:
close_loop_results = supply_chain_controller(time_ind=scale_iter_list, c_horizon=1, p_horizon=5, end_at=(3,6))

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint resource export
constraint transport export
constraint export
constraint transport capex
constraint transport network capex
constraint transport export network
constraint transport vopex
constraint transport network vopex
constraint transport fopex
constraint transport network fopex
constraint transport capacity UB no bin
constraint transport capacity LB no bin
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint transport capacity UB
constraint transport capacity LB
constraint fixed nameplate min production mode
constraint fixed nameplate max

## Fix planning horizon for a design problem

In [104]:
for t in model_record:
    print(f"Objective for {t}: {pyo_val(model_record[t].objective_cost_w_demand_penalty):.4f}")

Objective for (0, 0): 29144.9250
Objective for (0, 1): 27747.5250
Objective for (0, 2): 26350.3500
Objective for (0, 3): 24851.2500
Objective for (0, 4): 23352.1500
Objective for (0, 5): 22105.2750
Objective for (0, 6): 22105.2750
Objective for (1, 0): 21852.8250
Objective for (1, 1): 21852.8250
Objective for (1, 2): 21853.0500
Objective for (1, 3): 21853.0500
Objective for (1, 4): 21853.0500
Objective for (1, 5): 22105.2750
Objective for (1, 6): 22105.2750
Objective for (2, 0): 21852.8250
Objective for (2, 1): 21852.8250
Objective for (2, 2): 21853.0500
Objective for (2, 3): 21853.0500
Objective for (2, 4): 21853.0500
Objective for (2, 5): 22105.2750
Objective for (2, 6): 22105.2750
Objective for (3, 0): 21852.8250
Objective for (3, 1): 21852.8250
Objective for (3, 2): 18166.8250
Objective for (3, 3): 14564.2250
Objective for (3, 4): 10961.6250
Objective for (3, 5): 7460.7250
Objective for (3, 6): 3959.8250


In [105]:
model_record[(1,1)].P.pprint()

P : Production
    Size=1862, Index=locations*processes_full*scales_scheduling
    Key                                    : Lower : Value : Upper : Fixed : Stale : Domain
          ('loc1', 'com1_loc1_send', 0, 0) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 0, 1) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 0, 2) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 0, 3) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 0, 4) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 0, 5) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 0, 6) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('loc1', 'com1_loc1_send', 1, 0) :     0 :  30.0 :  None :  True :  True : NonNegativeReals
          ('l

In [106]:
(model_record[(1,2)].scales_scheduling.pprint())

scales_scheduling : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :   15 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 0)}


In [107]:
t = (3,6)

In [108]:
# print(*[(key, value) for key, value in close_loop_results[t]['P'].items() if value], sep='\n')

In [109]:
# write_iis(pyomo_model=model_record[t], iis_file_name=f'model_{t[0]}_{t[1]}', solver='gurobi')

In [110]:
# with open(f'model_{t[0]}_{t[1]}.ilp', 'rb') as f:
#     c = f.read()
# print(c)

In [111]:
from pyomo.environ import Constraint
constraint_names = [c.name for c in model_record[t].component_objects(ctype=Constraint)]

In [112]:
# constraint_names

In [113]:
# model_record[t].constraint_process_capex.pprint()

In [114]:
# model_record[t].constraint_process_fopex.pprint()

In [115]:
# model_record[t].constraint_process_vopex.pprint()

In [116]:
# model_record[t].constraint_process_incidental.pprint()

In [117]:
# model_record[t].constraint_location_capex.pprint()

In [118]:
# model_record[t].constraint_location_fopex.pprint()

In [119]:
# model_record[t].constraint_location_vopex.pprint()

In [120]:
# model_record[t].constraint_location_incidental.pprint()

In [121]:
# model_record[t].constraint_network_capex.pprint()

In [122]:
# model_record[t].constraint_network_fopex.pprint()

In [123]:
# model_record[t].constraint_network_vopex.pprint()

In [124]:
# model_record[t].constraint_network_incidental.pprint()

In [125]:
# model_record[t].constraint_storage_cost.pprint()

In [126]:
# model_record[t].constraint_storage_capex.pprint()

In [127]:
# model_record[t].constraint_storage_cost_location.pprint()

In [128]:
# model_record[t].constraint_storage_cost_network.pprint()

In [129]:
# model_record[t].constraint_nameplate_inventory.pprint()

In [130]:
# model_record[t].constraint_storage_max.pprint()

In [131]:
# model_record[t].constraint_production_mode.pprint()

In [132]:
# model_record[t].constraint_nameplate_production_certain_capacity.pprint()

In [133]:
# model_record[t].constraint_nameplate_production_varying_capacity.pprint()

In [134]:
# model_record[t].constraint_production_max.pprint()

In [135]:
# model_record[t].constraint_inventory_balance.pprint()

In [136]:
# model_record[t].constraint_resource_consumption_certain.pprint()

In [137]:
# model_record[t].constraint_resource_consumption_varying.pprint()

In [138]:
# model_record[t].constraint_resource_purchase_certain.pprint()

In [139]:
# model_record[t].constraint_resource_purchase_varying.pprint()

In [140]:
# model_record[t].constraint_location_production.pprint()

In [141]:
# model_record[t].constraint_location_discharge.pprint()

In [142]:
# model_record[t].constraint_location_consumption.pprint()

In [143]:
# model_record[t].constraint_location_purchase.pprint()

In [144]:
# model_record[t].constraint_network_production.pprint()

In [145]:
# model_record[t].constraint_network_discharge.pprint()

In [146]:
# model_record[t].constraint_network_consumption.pprint()

In [147]:
# model_record[t].constraint_network_purchase.pprint()

In [148]:
# model_record[t].constraint_inventory_network.pprint()

In [149]:
# model_record[t].constraint_resource_export.pprint()

In [150]:
# model_record[t].constraint_transport_export.pprint()

In [151]:
# model_record[t].constraint_export.pprint()

In [152]:
# model_record[t].constraint_transport_capex.pprint()

In [153]:
# model_record[t].constraint_transport_network_capex.pprint()

In [154]:
# model_record[t].constraint_transport_export_network.pprint()

In [155]:
# model_record[t].constraint_transport_vopex.pprint()

In [156]:
# model_record[t].constraint_transport_network_vopex.pprint()

In [157]:
# model_record[t].constraint_transport_fopex.pprint()

In [158]:
# model_record[t].constraint_transport_network_fopex.pprint()

In [159]:
# model_record[t].constraint_storage_facility.pprint()

In [160]:
# model_record[t].constraint_production_facility.pprint()

In [161]:
# model_record[t].constraint_min_production_facility.pprint()

In [162]:
# model_record[t].constraint_min_storage_facility.pprint()

In [163]:
# model_record[t].constraint_transport_capacity_UB.pprint()

In [164]:
# model_record[t].constraint_transport_capacity_LB.pprint()

In [165]:
# model_record[t].constraint_fixed_nameplate_min_production_mode.pprint()

In [166]:
# model_record[t].constraint_fixed_nameplate_max_production_mode.pprint()

In [167]:
# model_record[t].constraint_production_mode_binary.pprint()

In [168]:
# model_record[t].constraint_production_rate1.pprint()

In [169]:
# model_record[t].constraint_production_rate2.pprint()

In [170]:
# model_record[t].production_mode_switch.pprint()

In [171]:
# model_record[t].constraint_demand_penalty.pprint()

In [181]:
model_record[t].constraint_demand_penalty_location.pprint()

constraint_demand_penalty_location : total unmet demand at a location
    Size=14, Index=locations*resources_demand*{0, 1, 2, 3}, Active=True
    Key                      : Lower : Body                                                                                                                                                                                                                                 : Upper : Active
    ('loc1', 'com1_sold', 0) :   0.0 : Demand_penalty_location[loc1,com1_sold,0] - (Demand_penalty[loc1,com1_sold,0,2] + Demand_penalty[loc1,com1_sold,0,3] + Demand_penalty[loc1,com1_sold,0,4] + Demand_penalty[loc1,com1_sold,0,5] + Demand_penalty[loc1,com1_sold,0,6]) :   0.0 :   True
    ('loc1', 'com1_sold', 1) :   0.0 :                                                                                                                                                       Demand_penalty_location[loc1,com1_sold,1] - Demand_penalty[loc1,com1_sold,1,0] :   0.0 :   True
   

In [173]:
# model_record[t].constraint_demand_penalty_network.pprint()

In [174]:
# model_record[t].constraint_demand_penalty_cost.pprint()

In [175]:
# model_record[t].constraint_demand_penalty_cost_location.pprint()

In [176]:
# model_record[t].constraint_demand_penalty_cost_network.pprint()